# Semantic Kernel з інтеграцією сервера OpenBnB MCP

Цей нотатник демонструє, як використовувати Semantic Kernel з фактичним сервером OpenBnB MCP для пошуку реальних помешкань Airbnb за допомогою MCPStdioPlugin. Для доступу до LLM він використовує Microsoft Foundry. Щоб налаштувати змінні середовища, ви можете скористатися [уроком з налаштування](/00-course-setup/README.md)


## Імпорт необхідних пакетів


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Створення підключення MCP плагіна

Ми підключимося до [OpenBnB MCP server](https://github.com/openbnb-org/mcp-server-airbnb) за допомогою MCPStdioPlugin. Цей сервер забезпечує функціональність пошуку Airbnb через пакет @openbnb/mcp-server-airbnb.


## Створення клієнта

У цьому прикладі ми будемо використовувати Microsoft Foundry для доступу до нашої LLM. Переконайтеся, що ваші змінні середовища налаштовані правильно.


## Налаштування середовища

Налаштуйте параметри Azure OpenAI. Переконайтеся, що у вас встановлено такі змінні середовища:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Розуміння інтеграції OpenBnB MCP

Цей ноутбук підключається до **реального сервера OpenBnB MCP**, який надає фактичний функціонал пошуку Airbnb.

### Як це працює:

1. **MCPStdioPlugin**: Використовує стандартний ввід/вивід для комунікації з MCP сервером  
2. **Реальний NPM пакет**: Завантажує і запускає `@openbnb/mcp-server-airbnb` через npx  
3. **Живі дані**: Повертає фактичні дані про нерухомість Airbnb з їх API  
4. **Виявлення функцій**: Аґент автоматично знаходить доступні функції з MCP сервера  

### Доступні функції:

Сервер OpenBnB MCP зазвичай надає:  
- **search_listings** - Пошук нерухомості Airbnb за локацією та критеріями  
- **get_listing_details** - Отримання детальної інформації про конкретну нерухомість  
- **check_availability** - Перевірка доступності на конкретні дати  
- **get_reviews** - Отримання відгуків про нерухомість  
- **get_host_info** - Отримання інформації про господарів нерухомості  

### Вимоги:

- Встановлений **Node.js** у вашій системі  
- **Інтернет-з'єднання** для завантаження пакета MCP сервера  
- Доступний **NPX** (поставляється разом з Node.js)  

### Тестування з'єднання:

Ви можете вручну протестувати сервер MCP, запустивши:  
```bash
npx -y @openbnb/mcp-server-airbnb
```
  
Це завантажить та запустить сервер OpenBnB MCP, до якого Semantic Kernel потім підключається для отримання реальних даних Airbnb.


## Запуск Агента з OpenBnB MCP сервером

Тепер ми запустимо AI Агента, який підключається до OpenBnB MCP сервера для пошуку реальних помешкань Airbnb у Стокгольмі для 2 дорослих і 1 дитини. Не соромтеся змінювати список `user_inputs`, щоб змінити критерії пошуку.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Підсумок
Вітаємо! Ви успішно створили агент штучного інтелекту, який інтегрується з пошуком житла в реальному світі за допомогою протоколу Model Context Protocol (MCP):

## Використані технології:
- Semantic Kernel - для створення інтелектуальних агентів з Azure OpenAI
- Microsoft Foundry - для можливостей LLM та завершення чатів
- MCP (Model Context Protocol) - для стандартизованої інтеграції інструментів
- OpenBnB MCP Server - для реальної функціональності пошуку Airbnb
- Node.js/NPX - для запуску зовнішнього MCP сервера

## Чого ви навчилися:
- Інтеграція MCP: підключення агентів Semantic Kernel до зовнішніх MCP серверів
- Доступ до даних у реальному часі: пошук реальних об’єктів Airbnb через живі API
- Протокольна комунікація: використання stdio для спілкування між агентом і MCP сервером
- Виявлення функцій: автоматичне виявлення доступних функцій на MCP серверах
- Потокове отримання відповідей: захоплення та ведення журналу викликів функцій у реальному часі
- Відображення HTML: форматування відповідей агента із стилізованими таблицями та інтерактивними елементами

## Наступні кроки:
- Інтегрувати додаткові MCP сервери (погода, авіарейси, ресторани)
- Створити мультиагентну систему, що поєднує протоколи MCP і A2A
- Розробити власні MCP сервери для власних джерел даних
- Реалізувати постійну пам’ять розмов між сесіями
- Розгорнути агента на Azure Functions із оркестрацією MCP серверів
- Додати автентифікацію користувачів і можливості бронювання


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Застереження**:  
Цей документ було перекладено за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, просимо враховувати, що автоматичні переклади можуть містити помилки або неточності. Оригінальний документ рідною мовою слід вважати авторитетним джерелом. Для критично важливої інформації рекомендується користуватися професійним людським перекладом. Ми не несемо відповідальності за будь-які непорозуміння або неправильні тлумачення, що виникли внаслідок використання цього перекладу.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
